In [214]:
# import libraries
import pprint
import requests
import re
import pyjq
import json
from newspaper import Article
import pymongo
import datetime
from dateutil.rrule import rrule, MONTHLY
import math
import time

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
#clean this up for now
coll.drop()

start = 1960
end = 2020
years = []
for i in range(end-start):
    years.append(start + i)

api_key = 'mXe9rrak6WGZA7H2zzeGejAbh0LoCAzY'

for i in range(len(years)):    
    url_out = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&api-key=' + api_key
    r = requests .get(url_out)
    json_data = r.json()
    meta = pyjq.all('.response .meta',json_data)
    pages = math.ceil(meta[0]['hits']/10)
    print('Working on Year {0}. There are {1} page(s)'.format(years[i],pages))
    for p in range(pages):   
        url_in = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&page='+str(p)+'&api-key='+ api_key
        #print('Page {0} of {1}'.format(p,pages))
        r = requests .get(url_in)
        json_data = r.json()
        count = pyjq.all('.response .docs | length',json_data)[0]
        urls = pyjq.all('.response .docs[] .web_url',json_data)

        for j in range(count):
            toi_article = Article(urls[j], language="en")
            try:
                toi_article.download() 
            except:
                continue
            try:
                toi_article.parse() 
            except:
                continue
            try:
                toi_article.nlp() 
            except:
                continue
            coll.insert_one ( {'title':toi_article.title,
                   'text': toi_article.text,
                   'summary': toi_article.summary, 
                   'keywords': toi_article.keywords, 
                   'year':years[i],
                   'url': urls[j] } )
        time.sleep(6)

Working on Year 1960. There are 2 pages
Working on Year 1961. There are 1 pages
Working on Year 1962. There are 1 pages
Working on Year 1963. There are 1 pages
Working on Year 1964. There are 22 pages
Working on Year 1965. There are 1 pages
Working on Year 1966. There are 1 pages
Working on Year 1967. There are 1 pages
Working on Year 1968. There are 1 pages
Working on Year 1969. There are 2 pages
Working on Year 1970. There are 27 pages
Working on Year 1971. There are 20 pages
Working on Year 1972. There are 19 pages
Working on Year 1973. There are 27 pages
Working on Year 1974. There are 27 pages
Working on Year 1975. There are 29 pages
Working on Year 1976. There are 27 pages
Working on Year 1977. There are 32 pages
Working on Year 1978. There are 33 pages
Working on Year 1979. There are 34 pages
Working on Year 1980. There are 1 pages
Working on Year 1981. There are 42 pages
Working on Year 1982. There are 46 pages
Working on Year 1983. There are 35 pages
Working on Year 1984. Ther

ERROR:urllib3.connection:Certificate did not match expected hostname: fort-greene.thelocal.nytimes.com. Certificate: {'subject': ((('countryName', 'US'),), (('postalCode', '10018'),), (('stateOrProvinceName', 'New York'),), (('localityName', 'New York'),), (('streetAddress', '620 8th Ave'),), (('organizationName', 'The New York Times'),), (('organizationalUnitName', 'The New York Times'),), (('organizationalUnitName', 'Multi-Domain SSL'),), (('commonName', 'nytimes.com'),)), 'issuer': ((('countryName', 'GB'),), (('stateOrProvinceName', 'Greater Manchester'),), (('localityName', 'Salford'),), (('organizationName', 'COMODO CA Limited'),), (('commonName', 'COMODO RSA Organization Validation Secure Server CA'),)), 'version': 3, 'serialNumber': 'BFD6231BEA59AAD6CB566ED0878D60ED', 'notBefore': 'Nov 29 00:00:00 2018 GMT', 'notAfter': 'Jan 18 23:59:59 2020 GMT', 'subjectAltName': (('DNS', 'nytimes.com'), ('DNS', '*.api.dev.nytimes.com'), ('DNS', '*.api.nytimes.com'), ('DNS', '*.api.stg.nytimes

Working on Year 2010. There are 62 pages
Working on Year 2011. There are 51 pages
Working on Year 2012. There are 79 pages
Working on Year 2013. There are 48 pages
Working on Year 2014. There are 48 pages
Working on Year 2015. There are 48 pages
Working on Year 2016. There are 43 pages
Working on Year 2017. There are 45 pages
Working on Year 2018. There are 42 pages
Working on Year 2019. There are 42 pages


In [215]:
client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']

print('Total NYT articles {0}'.format(str(coll.count_documents({}))))

Total NYT articles 19340


In [ ]:
client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
pp = pprint.PrettyPrinter(indent=1,width=65)
for r in coll. find():
  print(pp .pformat (r))

In [216]:
import pandas as pd

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']

boom=[]
genX=[]
genY=[]
genZ=[]

cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1960, '$lte' : 1964 } } },{'$count': 'Article Count'}] )
boom = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1965, '$lte' : 1979 } } },{'$count': 'Article Count'}] )
genX = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1980, '$lte' : 1999 } } },{'$count': 'Article Count'}] )
genY = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 2000, '$lte' : 2019 } } },{'$count': 'Article Count'}] )
genZ = list(cursor)

df = pd.DataFrame(boom+genX+genY+genZ, index =['Boomers: 1960 to 1964',
                                               'Gen X:   1965 to 1979',
                                               'Gen Y:   1980 to 2000',
                                               'Gen Z:   2000 to 2019'],
                  columns =['Article Count']) 
display(df)
display(df.sum())

,Article Count
Boomers: 1960 to 1964,253
Gen X: 1965 to 1979,2745
Gen Y: 1980 to 2000,6793
Gen Z: 2000 to 2019,9549


Article Count    19340
dtype: int64

In [217]:
def get_dataframe(year):
    import pandas as pd
    cursor = coll.aggregate( [{ '$match': { 'year' : { '$eq' : year} } }] )
    df = pd.DataFrame(list(cursor))
    return df

In [219]:
display(get_dataframe(1981))

,_id,keywords,summary,text,title,url,year
0,5dbb472fbaff234d2b05f2f7,"[doctors, cases, delinquent, opinion, money, l...",To the Editor:The news of physicians defaultin...,To the Editor:\n\nThe news of physicians defau...,Opinion | WHEN DOCTORS DEFAULT ON STUDENT LOANS,https://www.nytimes.com/1981/12/24/opinion/l-w...,1981
1,5dbb4730baff234d2b05f2f8,"[doctors, school, failing, worst, loan, medica...",Many doctors are failing to pay back the Gover...,Many doctors are failing to pay back the Gover...,MANY DOCTORS FAILING TO PAY STUDENT LOANS,https://www.nytimes.com/1981/12/07/us/many-doc...,1981
2,5dbb4731baff234d2b05f2f9,"[effect, program, restriction, family, month, ...","Starting next month, a student attending the a...","Starting next month, a student attending the a...",RESTRICTION ON STUDENT LOANS IS TO TAKE EFFECT...,https://www.nytimes.com/1981/09/06/us/restrict...,1981
3,5dbb4732baff234d2b05f2fa,"[sought, raise, secretary, needs, house, educa...",The compromise liberalized both proposals by r...,The compromise liberalized both proposals by r...,"CONFEREES RAISE LIMIT TO $30,000 FOR NEEDS TES...",https://www.nytimes.com/1981/07/22/us/conferee...,1981
4,5dbb4734baff234d2b05f2fb,"[program, loopholes, middleincome, opinion, tu...",Credit President Reagan with sicking his budge...,Credit President Reagan with sicking his budge...,Opinion | Plugging the Student Loan Loopholes,https://www.nytimes.com/1981/04/21/opinion/plu...,1981
5,5dbb4737baff234d2b05f2fc,"[fees, doctors, program, opinion, residencies,...",Assuming that HEAL and bank-loan rates will av...,If the Senate bill is accepted by the House - ...,Opinion | CUTTING MEDICAL-STUDENT LOANS MAY DR...,https://www.nytimes.com/1981/07/20/opinion/cut...,1981
6,5dbb4738baff234d2b05f2fd,"[program, subscriberssee, opinion, timesmachin...",TimesMachine is an exclusive benefit for home ...,TimesMachine is an exclusive benefit for home ...,Opinion | REPAIR TIPS FOR THE STUDENT LOAN PRO...,https://www.nytimes.com/1981/05/20/opinion/l-r...,1981
7,5dbb4739baff234d2b05f2fe,"[gsl, annual, 40000, benefits, opinion, imposs...",To the Editor:I am appalled at the responses t...,To the Editor:\n\nI am appalled at the respons...,Opinion | IF THE MIDDLE CLASS LOSES STUDENT LOANS,https://www.nytimes.com/1981/05/13/opinion/l-i...,1981
8,5dbb473abaff234d2b05f2ff,"[plan, committee, panel, alter, bars, reagan, ...",The House Appropriations Committee today sough...,The House Appropriations Committee today sough...,Panel Bars Reagan Plan To Alter Student Loans,https://www.nytimes.com/1981/05/13/us/panel-ba...,1981
9,5dbb473bbaff234d2b05f300,"[program, save, opinion, lets, needy, lenders,...",The principal attraction was the abolition of ...,The fact is that the increased borrowing was m...,Opinion | LET'S NOT SAVE THE STUDENT LOAN PROG...,https://www.nytimes.com/1981/02/05/opinion/l-l...,1981
